# 🔌 CapAssigner - Capacitor Network Synthesis Tool

**Ejecuta CapAssigner directamente en Google Colab**

Esta aplicación te permite diseñar redes de capacitores para alcanzar una capacitancia equivalente objetivo.

## Instrucciones

1. **Paso 1**: Ejecuta la celda de instalación (~2 minutos)
2. **Paso 2**: Inicia la aplicación Streamlit
3. **Paso 3**: Crea el túnel público
4. **Paso 4**: Accede a la URL generada

---

In [ ]:
#@title ⚙️ Paso 1: Instalación de dependencias (~2 minutos) { display-mode:"form" }
import subprocess, sys, os
from IPython.display import Markdown, display, clear_output

REPO_URL = "https://github.com/elloza/CapAssigner.git"
REPO_DIR = "/content/CapAssigner"
PIP_LOG  = "/content/pip_install.log"

def step(msg):
    display(Markdown(f"⏳ **{msg}…**"))

def run(cmd, log_file=None):
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    out, _ = proc.communicate()
    if log_file:
        with open(log_file, "a") as f: f.write(out)
    if proc.returncode:
        raise subprocess.CalledProcessError(proc.returncode, cmd)

# Clonar repo
if not os.path.isdir(REPO_DIR):
    step("Clonando CapAssigner")
    subprocess.run(["git", "clone", "-q", REPO_URL, REPO_DIR], check=True)
else:
    step("Actualizando repositorio")
    subprocess.run(["git", "-C", REPO_DIR, "pull", "-q"], check=True)
clear_output(wait=True)

# Instalar dependencias
req = os.path.join(REPO_DIR, "requirements.txt")
step("Instalando dependencias Python")
try:
    run([sys.executable, "-m", "pip", "install", "-q", "-r", req], log_file=PIP_LOG)
    clear_output(wait=True)
    display(Markdown("✅ **Instalación completada!** Pasa al **Paso 2**."))
except:
    clear_output(wait=True)
    display(Markdown("❌ **Error.** Ejecuta: `!tail -n 30 /content/pip_install.log`"))

In [ ]:
#@title 🚀 Paso 2: Iniciar Streamlit { display-mode:"form" }
import subprocess, time, requests, sys
from IPython.display import Markdown, display, clear_output

APP_DIR = "/content/CapAssigner"
PORT = "8501"

subprocess.call(["pkill", "-f", "streamlit"], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

display(Markdown("⏳ **Arrancando Streamlit…**"))
log = open("/content/st.log", "w")
proc = subprocess.Popen(
    [sys.executable, "-m", "streamlit", "run", "app.py",
     "--server.headless", "true", "--server.port", PORT],
    cwd=APP_DIR, stdout=log, stderr=subprocess.STDOUT)

# Esperar a que arranque
for _ in range(60):
    try:
        if requests.get(f"http://localhost:{PORT}").status_code == 200:
            break
    except: pass
    time.sleep(1)

clear_output(wait=True)
try:
    if requests.get(f"http://localhost:{PORT}").status_code == 200:
        display(Markdown("✅ **Streamlit listo!** Pasa al **Paso 3**."))
    else:
        display(Markdown("❌ **Error.** Ejecuta: `!tail -n 40 /content/st.log`"))
except:
    display(Markdown("❌ **Error.** Ejecuta: `!tail -n 40 /content/st.log`"))

In [ ]:
#@title 🌐 Paso 3: Crear túnel público { display-mode:"form" }
import subprocess, time, re, os
from IPython.display import Markdown, display, clear_output

display(Markdown("⏳ **Descargando cloudflared…**"))
if not os.path.isfile("/content/cloudflared"):
    subprocess.run(["wget", "-q",
        "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64",
        "-O", "/content/cloudflared"], check=True)
    subprocess.run(["chmod", "+x", "/content/cloudflared"], check=True)
clear_output(wait=True)

display(Markdown("⏳ **Iniciando túnel…**"))
log_f = open("/content/cf.log", "w")
subprocess.Popen(["/content/cloudflared", "tunnel",
    "--url", "http://localhost:8501", "--no-autoupdate"],
    stdout=log_f, stderr=subprocess.STDOUT)
time.sleep(10)
log_f.close()
clear_output(wait=True)

with open("/content/cf.log") as f:
    m = re.search(r"https://[^\s]*\.trycloudflare\.com", f.read())
url = m.group(0) if m else ""
open("/content/app_url.txt", "w").write(url)

if url:
    display(Markdown("✅ **Túnel listo!** Ejecuta el **Paso 4**."))
else:
    display(Markdown("❌ **Error.** Ejecuta: `!cat /content/cf.log`"))

In [ ]:
#@title 🔗 Paso 4: Acceder a CapAssigner { display-mode:"form" }
from IPython.display import Markdown, display

try:
    url = open("/content/app_url.txt").read().strip()
    if url:
        display(Markdown(f"""## ✅ CapAssigner está listo!

### 👉 [Haz clic aquí para abrir la aplicación]({url})

**URL:** `{url}`

*La primera carga puede tardar unos segundos.*"""))
    else:
        display(Markdown("❌ **Ejecuta el Paso 3 primero.**"))
except:
    display(Markdown("❌ **Ejecuta el Paso 3 primero.**"))

---

## 🔧 Solución de problemas

Si algo falla, ejecuta estas celdas para depurar:

```python
# Log de Streamlit
!tail -n 50 /content/st.log

# Log de Cloudflare
!cat /content/cf.log

# Log de instalación
!tail -n 50 /content/pip_install.log
```

---

*CapAssigner - Built with ❤️ using Streamlit, NetworkX, SchemDraw, and NumPy*